In [1]:
from mem_sim import *

# Test Set 1: Sequential Access Pattern
This test simulates sequential memory access (e.g., array traversal).
- **Pattern**: Sequential addresses with stride
- **Size**: 600 accesses
- **Expected behavior**: High TLB hit rate after initial warm-up

In [4]:
import random

# Test Set 1: Sequential Access Pattern
# Configuration
PAGE_SIZE = 4096
NUM_TLB_ENTRIES = 16
NUM_FRAMES = 64
POLICY = 'LRU'
NUM_ACCESSES = 600

# Generate sequential access trace
test1_trace = []
base_address = 0x1000000  # 16MB base
stride = 64  # 64 bytes stride (cache line size)

for i in range(NUM_ACCESSES):
    address = base_address + (i * stride)
    test1_trace.append(address)

# Save trace to file
with open('tests/test1_sequential.trace', 'w') as f:
    for addr in test1_trace:
        f.write(f"{addr}\n")

print(f"Test 1 generated: {len(test1_trace)} sequential accesses")
print(f"Address range: {hex(min(test1_trace))} - {hex(max(test1_trace))}")
print(f"Pages accessed: ~{(max(test1_trace) - min(test1_trace)) // PAGE_SIZE + 1}")

Test 1 generated: 600 sequential accesses
Address range: 0x1000000 - 0x10095c0
Pages accessed: ~10


In [ ]:
# Run Test 1 with LRU
print("=" * 70)
print("TEST 1: SEQUENTIAL ACCESS PATTERN - LRU")
print("=" * 70)

sim_test1_lru = MemorySimulator(
    page_size=PAGE_SIZE,
    num_tlb_entries=NUM_TLB_ENTRIES,
    num_frames=NUM_FRAMES,
    replacement_policy='LRU',
    debug=False
)testing

for addr in test1_trace:
    sim_test1_lru.access_memory(addr)

sim_test1_lru.print_statistics()

# Save expected output
with open('tests/test1_sequential_lru.expected', 'w') as f:
    f.write("=" * 60 + "\n")
    f.write("SIMULADOR DE MEMÓRIA - Estatísticas de Acesso\n")
    f.write("=" * 60 + "\n")
    f.write(f"Política de Substituição:   LRU\n")
    f.write(f"Tamanho da Página:          {PAGE_SIZE} bytes\n")
    f.write(f"Entradas na TLB:            {NUM_TLB_ENTRIES}\n")
    f.write(f"Número de Frames:           {NUM_FRAMES}\n")
    f.write("-" * 60 + "\n")
    f.write(f"TLB Hits:                   {sim_test1_lru.tlb_hits}\n")
    f.write(f"TLB Misses:                 {sim_test1_lru.tlb_misses}\n")
    f.write(f"Page Faults:                {sim_test1_lru.page_faults}\n")
    f.write("=" * 60 + "\n")

TEST 1: SEQUENTIAL ACCESS PATTERN - LRU
SIMULADOR DE MEMÓRIA - Estatísticas de Acesso
Política de Substituição:   LRU
Tamanho da Página:          4096 bytes
Entradas na TLB:            16
Número de Frames:           64
------------------------------------------------------------
TLB Hits:                   590
TLB Misses:                 10
Page Faults:                10


In [6]:
# Run Test 1 with Second Chance
print("\n" + "=" * 70)
print("TEST 1: SEQUENTIAL ACCESS PATTERN - SECOND CHANCE")
print("=" * 70)

sim_test1_sc = MemorySimulator(
    page_size=PAGE_SIZE,
    num_tlb_entries=NUM_TLB_ENTRIES,
    num_frames=NUM_FRAMES,
    replacement_policy='SecondChance',
    debug=False
)

for addr in test1_trace:
    sim_test1_sc.access_memory(addr)

sim_test1_sc.print_statistics()

# Save expected output
with open('tests/test1_sequential_secondchance.expected', 'w') as f:
    f.write("=" * 60 + "\n")
    f.write("SIMULADOR DE MEMÓRIA - Estatísticas de Acesso\n")
    f.write("=" * 60 + "\n")
    f.write(f"Política de Substituição:   SecondChance\n")
    f.write(f"Tamanho da Página:          {PAGE_SIZE} bytes\n")
    f.write(f"Entradas na TLB:            {NUM_TLB_ENTRIES}\n")
    f.write(f"Número de Frames:           {NUM_FRAMES}\n")
    f.write("-" * 60 + "\n")
    f.write(f"TLB Hits:                   {sim_test1_sc.tlb_hits}\n")
    f.write(f"TLB Misses:                 {sim_test1_sc.tlb_misses}\n")
    f.write(f"Page Faults:                {sim_test1_sc.page_faults}\n")
    f.write("=" * 60 + "\n")


TEST 1: SEQUENTIAL ACCESS PATTERN - SECOND CHANCE
SIMULADOR DE MEMÓRIA - Estatísticas de Acesso
Política de Substituição:   SecondChance
Tamanho da Página:          4096 bytes
Entradas na TLB:            16
Número de Frames:           64
------------------------------------------------------------
TLB Hits:                   590
TLB Misses:                 10
Page Faults:                10


# Test Set 2: Random Access Pattern
This test simulates random memory access with locality.
- **Pattern**: Random addresses within a working set, with some repeated accesses
- **Size**: 800 accesses
- **Expected behavior**: More page faults, lower TLB hit rate than sequential

In [7]:
# Test Set 2: Random Access with Locality
NUM_ACCESSES_2 = 800
WORKING_SET_PAGES = 100  # Access within 100 pages
LOCALITY_FACTOR = 0.3  # 30% chance to re-access recent page

test2_trace = []
base_address = 0x2000000  # 32MB base
recent_pages = []

random.seed(42)  # For reproducibility

for i in range(NUM_ACCESSES_2):
    if recent_pages and random.random() < LOCALITY_FACTOR:
        # Re-access a recent page (temporal locality)
        page_num = random.choice(recent_pages[-10:])  # From last 10 pages
    else:
        # Access a random page within working set
        page_num = random.randint(0, WORKING_SET_PAGES - 1)
    
    # Random offset within page
    offset = random.randint(0, PAGE_SIZE - 1) & ~0x3  # Align to 4 bytes
    address = base_address + (page_num * PAGE_SIZE) + offset
    test2_trace.append(address)
    
    if page_num not in recent_pages[-10:]:
        recent_pages.append(page_num)

# Save trace to file
with open('tests/test2_random.trace', 'w') as f:
    for addr in test2_trace:
        f.write(f"{addr}\n")

print(f"Test 2 generated: {len(test2_trace)} random accesses with locality")
print(f"Address range: {hex(min(test2_trace))} - {hex(max(test2_trace))}")
unique_pages = len(set(addr // PAGE_SIZE for addr in test2_trace))
print(f"Unique pages accessed: {unique_pages}")

Test 2 generated: 800 random accesses with locality
Address range: 0x2000244 - 0x2063f14
Unique pages accessed: 98


In [8]:
# Run Test 2 with LRU
print("=" * 70)
print("TEST 2: RANDOM ACCESS WITH LOCALITY - LRU")
print("=" * 70)

sim_test2_lru = MemorySimulator(
    page_size=PAGE_SIZE,
    num_tlb_entries=NUM_TLB_ENTRIES,
    num_frames=NUM_FRAMES,
    replacement_policy='LRU',
    debug=False
)

for addr in test2_trace:
    sim_test2_lru.access_memory(addr)

sim_test2_lru.print_statistics()

# Save expected output
with open('tests/test2_random_lru.expected', 'w') as f:
    f.write("=" * 60 + "\n")
    f.write("SIMULADOR DE MEMÓRIA - Estatísticas de Acesso\n")
    f.write("=" * 60 + "\n")
    f.write(f"Política de Substituição:   LRU\n")
    f.write(f"Tamanho da Página:          {PAGE_SIZE} bytes\n")
    f.write(f"Entradas na TLB:            {NUM_TLB_ENTRIES}\n")
    f.write(f"Número de Frames:           {NUM_FRAMES}\n")
    f.write("-" * 60 + "\n")
    f.write(f"TLB Hits:                   {sim_test2_lru.tlb_hits}\n")
    f.write(f"TLB Misses:                 {sim_test2_lru.tlb_misses}\n")
    f.write(f"Page Faults:                {sim_test2_lru.page_faults}\n")
    f.write("=" * 60 + "\n")

TEST 2: RANDOM ACCESS WITH LOCALITY - LRU
SIMULADOR DE MEMÓRIA - Estatísticas de Acesso
Política de Substituição:   LRU
Tamanho da Página:          4096 bytes
Entradas na TLB:            16
Número de Frames:           64
------------------------------------------------------------
TLB Hits:                   325
TLB Misses:                 475
Page Faults:                226


In [9]:
# Run Test 2 with Second Chance
print("\n" + "=" * 70)
print("TEST 2: RANDOM ACCESS WITH LOCALITY - SECOND CHANCE")
print("=" * 70)

sim_test2_sc = MemorySimulator(
    page_size=PAGE_SIZE,
    num_tlb_entries=NUM_TLB_ENTRIES,
    num_frames=NUM_FRAMES,
    replacement_policy='SecondChance',
    debug=False
)

for addr in test2_trace:
    sim_test2_sc.access_memory(addr)

sim_test2_sc.print_statistics()

# Save expected output
with open('tests/test2_random_secondchance.expected', 'w') as f:
    f.write("=" * 60 + "\n")
    f.write("SIMULADOR DE MEMÓRIA - Estatísticas de Acesso\n")
    f.write("=" * 60 + "\n")
    f.write(f"Política de Substituição:   SecondChance\n")
    f.write(f"Tamanho da Página:          {PAGE_SIZE} bytes\n")
    f.write(f"Entradas na TLB:            {NUM_TLB_ENTRIES}\n")
    f.write(f"Número de Frames:           {NUM_FRAMES}\n")
    f.write("-" * 60 + "\n")
    f.write(f"TLB Hits:                   {sim_test2_sc.tlb_hits}\n")
    f.write(f"TLB Misses:                 {sim_test2_sc.tlb_misses}\n")
    f.write(f"Page Faults:                {sim_test2_sc.page_faults}\n")
    f.write("=" * 60 + "\n")


TEST 2: RANDOM ACCESS WITH LOCALITY - SECOND CHANCE
SIMULADOR DE MEMÓRIA - Estatísticas de Acesso
Política de Substituição:   SecondChance
Tamanho da Página:          4096 bytes
Entradas na TLB:            16
Número de Frames:           64
------------------------------------------------------------
TLB Hits:                   325
TLB Misses:                 475
Page Faults:                222


# Test Set 3: GEMM-like Loop Pattern (Bonus)
This test simulates matrix multiplication access pattern.
- **Pattern**: Nested loops accessing 2D arrays (matrix multiplication)
- **Size**: 1000 accesses
- **Expected behavior**: Mixed locality with both spatial and temporal patterns

In [10]:
# Test Set 3: GEMM-like (Matrix Multiplication) Access Pattern
NUM_ACCESSES_3 = 1000
MATRIX_SIZE = 32  # 32x32 matrices
ELEMENT_SIZE = 8  # 8 bytes per element (double)

test3_trace = []
base_A = 0x3000000  # Matrix A base
base_B = 0x3100000  # Matrix B base
base_C = 0x3200000  # Matrix C base

# Simulate C = A * B
# for i in range(N):
#     for j in range(N):
#         for k in range(N):
#             C[i][j] += A[i][k] * B[k][j]

accesses = 0
for i in range(MATRIX_SIZE):
    for j in range(MATRIX_SIZE):
        for k in range(min(MATRIX_SIZE, (NUM_ACCESSES_3 - accesses) // 3)):
            # Access A[i][k]
            addr_a = base_A + (i * MATRIX_SIZE + k) * ELEMENT_SIZE
            test3_trace.append(addr_a)
            
            # Access B[k][j]
            addr_b = base_B + (k * MATRIX_SIZE + j) * ELEMENT_SIZE
            test3_trace.append(addr_b)
            
            # Access C[i][j]
            addr_c = base_C + (i * MATRIX_SIZE + j) * ELEMENT_SIZE
            test3_trace.append(addr_c)
            
            accesses += 3
            if accesses >= NUM_ACCESSES_3:
                break
        if accesses >= NUM_ACCESSES_3:
            break
    if accesses >= NUM_ACCESSES_3:
        break

# Save trace to file
with open('tests/test3_gemm.trace', 'w') as f:
    for addr in test3_trace:
        f.write(f"{addr}\n")

print(f"Test 3 generated: {len(test3_trace)} GEMM-like accesses")
print(f"Matrix size: {MATRIX_SIZE}x{MATRIX_SIZE}")
unique_pages = len(set(addr // PAGE_SIZE for addr in test3_trace))
print(f"Unique pages accessed: {unique_pages}")

Test 3 generated: 999 GEMM-like accesses
Matrix size: 32x32
Unique pages accessed: 4


In [11]:
# Run Test 3 with LRU
print("=" * 70)
print("TEST 3: GEMM-LIKE PATTERN - LRU")
print("=" * 70)

sim_test3_lru = MemorySimulator(
    page_size=PAGE_SIZE,
    num_tlb_entries=NUM_TLB_ENTRIES,
    num_frames=NUM_FRAMES,
    replacement_policy='LRU',
    debug=False
)

for addr in test3_trace:
    sim_test3_lru.access_memory(addr)

sim_test3_lru.print_statistics()

# Save expected output
with open('tests/test3_gemm_lru.expected', 'w') as f:
    f.write("=" * 60 + "\n")
    f.write("SIMULADOR DE MEMÓRIA - Estatísticas de Acesso\n")
    f.write("=" * 60 + "\n")
    f.write(f"Política de Substituição:   LRU\n")
    f.write(f"Tamanho da Página:          {PAGE_SIZE} bytes\n")
    f.write(f"Entradas na TLB:            {NUM_TLB_ENTRIES}\n")
    f.write(f"Número de Frames:           {NUM_FRAMES}\n")
    f.write("-" * 60 + "\n")
    f.write(f"TLB Hits:                   {sim_test3_lru.tlb_hits}\n")
    f.write(f"TLB Misses:                 {sim_test3_lru.tlb_misses}\n")
    f.write(f"Page Faults:                {sim_test3_lru.page_faults}\n")
    f.write("=" * 60 + "\n")

TEST 3: GEMM-LIKE PATTERN - LRU
SIMULADOR DE MEMÓRIA - Estatísticas de Acesso
Política de Substituição:   LRU
Tamanho da Página:          4096 bytes
Entradas na TLB:            16
Número de Frames:           64
------------------------------------------------------------
TLB Hits:                   995
TLB Misses:                 4
Page Faults:                4


In [12]:
# Run Test 3 with Second Chance
print("\n" + "=" * 70)
print("TEST 3: GEMM-LIKE PATTERN - SECOND CHANCE")
print("=" * 70)

sim_test3_sc = MemorySimulator(
    page_size=PAGE_SIZE,
    num_tlb_entries=NUM_TLB_ENTRIES,
    num_frames=NUM_FRAMES,
    replacement_policy='SecondChance',
    debug=False
)

for addr in test3_trace:
    sim_test3_sc.access_memory(addr)

sim_test3_sc.print_statistics()

# Save expected output
with open('tests/test3_gemm_secondchance.expected', 'w') as f:
    f.write("=" * 60 + "\n")
    f.write("SIMULADOR DE MEMÓRIA - Estatísticas de Acesso\n")
    f.write("=" * 60 + "\n")
    f.write(f"Política de Substituição:   SecondChance\n")
    f.write(f"Tamanho da Página:          {PAGE_SIZE} bytes\n")
    f.write(f"Entradas na TLB:            {NUM_TLB_ENTRIES}\n")
    f.write(f"Número de Frames:           {NUM_FRAMES}\n")
    f.write("-" * 60 + "\n")
    f.write(f"TLB Hits:                   {sim_test3_sc.tlb_hits}\n")
    f.write(f"TLB Misses:                 {sim_test3_sc.tlb_misses}\n")
    f.write(f"Page Faults:                {sim_test3_sc.page_faults}\n")
    f.write("=" * 60 + "\n")


TEST 3: GEMM-LIKE PATTERN - SECOND CHANCE
SIMULADOR DE MEMÓRIA - Estatísticas de Acesso
Política de Substituição:   SecondChance
Tamanho da Página:          4096 bytes
Entradas na TLB:            16
Número de Frames:           64
------------------------------------------------------------
TLB Hits:                   995
TLB Misses:                 4
Page Faults:                4


# Summary of Test Sets

## Test 1: Sequential Access (600 accesses)
- **Pattern**: Linear sequential with stride
- **Use case**: Array/vector traversal
- **Expected**: High TLB hit rate, few page faults after warm-up

## Test 2: Random with Locality (800 accesses)  
- **Pattern**: Random with 30% temporal locality
- **Use case**: Hash table or tree traversal
- **Expected**: Moderate TLB hits, more page faults than sequential

## Test 3: GEMM Loop Pattern (1000 accesses)
- **Pattern**: Nested loop matrix access (i-j-k)
- **Use case**: Matrix multiplication
- **Expected**: Mixed behavior with spatial and temporal locality

All test files have been generated and saved in the `tests/` directory:
- Trace files: `.trace`
- Expected outputs: `.expected`